# XGBoost training
This is an auto-generated notebook. To reproduce these results, attach this notebook to the **10-3-ML-Cluster** cluster and rerun it.
- Compare trials in the [MLflow experiment](#mlflow/experiments/406583024052808/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Navigate to the parent notebook [here](#notebook/406583024052798) (If you launched the AutoML experiment using the Experiments UI, this link isn't very useful.)
- Clone this notebook into your project folder by selecting **File > Clone** in the notebook toolbar.

Runtime Version: _10.3.x-cpu-ml-scala2.12_

In [ ]:
import mlflow
import databricks.automl_runtime

# Use MLflow to track experiments
mlflow.set_experiment("/Users/noah.gift@gmail.com/databricks_automl/label_news_articles_csv-2022_03_12-15_38")

target_col = "label"

: 

## Load Data

In [ ]:
from mlflow.tracking import MlflowClient
import os
import uuid
import shutil
import pandas as pd

# Create temp directory to download input data from MLflow
input_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(input_temp_dir)


# Download the artifact and read it into a pandas DataFrame
input_client = MlflowClient()
input_data_path = input_client.download_artifacts("c2dfe80b419d4a8dbc88a90e3274369a", "data", input_temp_dir)

df_loaded = pd.read_parquet(os.path.join(input_data_path, "training_data"))
# Delete the temp data
shutil.rmtree(input_temp_dir)

# Preview data
df_loaded.head(5)

Out[2]:

,author,published,title,text,language,site_url,main_img_url,type,title_without_stopwords,text_without_stopwords,hasImage,label
0,noreply@blogger.com (Der Postillon),2016-10-27T18:05:26.351+03:00,umfrage ceta würde enorm an zustimmung gewinne...,samstag oktober autofahrer entlarvt geheimen...,german,der-postillon.com,https://2.bp.blogspot.com/-0mdp0nZiwMI/UYwYvex...,bs,morgen neu kiosk postillon sonntag,montag oktober gruselig fc bayern verkleidet s...,1.0,Fake
1,noreply@blogger.com (Der Postillon),2016-10-28T10:34:13.418+03:00,newsticker,freitag oktober alle machen jagd auf ihn hor...,german,der-postillon.com,https://2.bp.blogspot.com/-cU9TKlQVRRg/WBSAnXO...,bs,autofahrer entlarvt geheimen zahlentrick mit d...,rausgewunken betrunkener autofahrer reagiert f...,1.0,Fake
2,noreply@blogger.com (Der Postillon),2016-10-29T16:01:22.811+03:00,morgen neu am kiosk postillon am sonntag,montag oktober gruselig fc bayern verkleidet...,german,der-postillon.com,https://1.bp.blogspot.com/-M9-qQTag-wM/Vi-k87F...,bs,alle machen jagd auf ihn horrorclown traut sic...,sonntag oktober sonntagsfrage machen sie heute...,1.0,Fake
3,noreply@blogger.com (Der Postillon),2016-10-29T15:09:00.000+03:00,autofahrer entlarvt geheimen zahlentrick mit d...,rausgewunken betrunkener autofahrer reagiert f...,german,der-postillon.com,https://1.bp.blogspot.com/-dWo3ZWGvWpE/WBMxjRv...,bs,gruselig fc bayern verkleidet sich zu hallowee...,sonntag oktober wegen ruhestörung gerufene pol...,1.0,Fake
4,noreply@blogger.com (Der Postillon),2016-10-29T16:01:28.149+03:00,alle machen jagd auf ihn horrorclown traut sic...,sonntag oktober sonntagsfrage was machen sie...,german,der-postillon.com,https://1.bp.blogspot.com/-O2G8EiK3_G0/WBc6lrJ...,bs,newsticker,montag oktober rufschädigung brauerei verbiete...,1.0,Fake


In [ ]:
df_loaded.head(1).to_dict()

Out[42]: {'author': {0: 'noreply@blogger.com (Der Postillon)'},
 'published': {0: '2016-10-27T18:05:26.351+03:00'},
 'title': {0: 'umfrage ceta würde enorm an zustimmung gewinnen wenn eu kanadische cannabisstandards übernehmen müsste'},
 'text': {0: 'samstag oktober autofahrer entlarvt geheimen zahlentrick mit dem sich jeder blitzer überlisten lässt berlin archiv ein berliner autofahrer hat einen geheimen trick entdeckt mit dem sich radarfallen und blitzer auf autobahnen zu prozent überlisten lassen erste praxistests scheinen die theorie des hobbyphysikers zu bestätigen seine zahlenbasierte methode soll für jeden autofahrer innerhalb von minuten erlernbar sein seit ist jörg haffke jeden tag auf deutschen autobahnen unterwegs was den berufspendler oft teuer zu stehen kommt früher verging kaum eine woche ohne dass ich ein knöllchen im briefkasten hatte nur weil ich mal wieder in eine radarfalle geraten war klagt der jährige die verschiedensten gegenmittel habe er schon ausprobiert reflektierende cds am innenspiegel abgedecktes kennzeichen oder blitzwarnerapps nichts hatte auf dauer wirklich erfolg berichtet der familienvater er hat den code geknackt jörg haffke haffke ist jedoch nicht nur ein cleverer bastler sondern auch ein guter beobachter irgendwann fiel mir auf dass neben den autobahnen immer wieder so merkwürdige schilder mit zahlen drauf versteckt sind zwei jahre und exceltabellen später ist sich haffke nun sicher den geheimen algorithmus hinter den kryptischen blechtafeln endlich entlarvt zu haben wenn man ein tempo mit einem tachowert fährt der in etwa der zahl auf dem letzten gesichteten schild entspricht oder darunter liegt verhindert man ein auslösen des nächsten blitzers man ist praktisch unsichtbar welche physikalische kraft diese offensichtliche fernwirkung herbeiführt sei ihm zwar nicht ganz klar vielleicht gibt es ja doch etwas das größer ist als wir das ergebnis gebe ihm aber recht keine knöllchen in den letzten sechs monaten erklärt er stolz nun will haffke seine erfahrungen an andere autofahrer weitergeben und hat dazu die standorte der knapp fünf millionen bisher identifizierten zahlenschilder in einem einzigartigen nachschlagewerk verzeichnet jörg haffke zahlenschilder in deutschland die geheimen zeichen mit denen sich radarfallen überlisten lassen s selbstverlag eigenheim bep kojote fotos shutterstock hinweis erstmals erschienen am artikel teilen'},
 'language': {0: 'german'},
 'site_url': {0: 'der-postillon.com'},
 'main_img_url': {0: 'https://2.bp.blogspot.com/-0mdp0nZiwMI/UYwYvexmW2I/AAAAAAAAVQM/7C_X5WRE_mQ/w1200-h630-p-nu/Edison-Stock-Ticker.jpg'},
 'type': {0: 'bs'},
 'title_without_stopwords': {0: 'morgen neu kiosk postillon sonntag'},
 'text_without_stopwords': {0: 'montag oktober gruselig fc bayern verkleidet sich zu halloween als hsv münchen dpo ob sich ihre fans von diesem schreck jemals erholen wie sport postillon soeben exklusiv erfuhr wird der fc bayern münchen heute auf der vereinsinternen halloweenfeier geschlossen als hsv verkleidet auftreten für den gruselspaß hat sich die mannschaft einen kompletten trikotsatz des künftigen zweitligisten besorgt wie schaurig die kostümierung auf den eigenen anhang wirkt zeigt eine szene von heute mittag als die münchner für ein mannschaftsfoto posierten um den aufwendigen halloweenstreich zu dokumentieren drei bayernfans wurden zufällig zeugen der aktion mannschaftsfotograf uli höpfer die wurden kreidebleich zwei rannten schreiend weg der dritte begab sich fötusstellung und wippte wimmernd hin und er wird derzeit psychologisch betreut heute abend geht es dann zu einer vereinsinternen halloweenfeier einem bekannten münchener szeneclub die mannschaft bereitet sich bereits intensiv auf die rolle als hsv vor wir trinken jeder schon mal zwei maß und ein paar kurze bevor wir überhaupt auf die feier gehen erklärt thomas müller dessen idee die gruselige maskerade war dadurch gleichen wir unseren bewegungsablauf den des hsv auch einige running gags haben sich die 

### Select supported columns
Select only the columns that are supported. This allows us to train a model that can predict on a dataset that has extra columns that are not used in training.
`[]` are dropped in the pipelines. See the Alerts tab of the AutoML Experiment page for details on why these columns are dropped.

In [ ]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
supported_cols = ["text_without_stopwords", "published", "language", "main_img_url", "site_url", "hasImage", "title_without_stopwords", "text", "title", "type", "author"]
col_selector = ColumnSelector(supported_cols)

## Preprocessors

In [ ]:
transformers = []

### Categorical columns

#### Low-cardinality categoricals
Convert each low-cardinality categorical column into multiple binary columns through one-hot encoding.
For each input categorical column (string or numeric), the number of output columns is equal to the number of unique values in the input column.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(handle_unknown="ignore")

transformers.append(("onehot", one_hot_encoder, ["published", "language", "site_url", "hasImage", "title", "title_without_stopwords", "text_without_stopwords"]))

#### Medium-cardinality categoricals
Convert each medium-cardinality categorical column into a numerical representation.
Each string column is hashed to 1024 float columns.
Each numeric column is imputed with zeros.

In [ ]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

for feature in ["text", "main_img_url"]:
    hash_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(missing_values=None, strategy="constant", fill_value="")),
        (f"{feature}_hasher", FeatureHasher(n_features=1024, input_type="string"))])
    transformers.append((f"{feature}_hasher", hash_transformer, [feature]))

### Text features
Convert each feature to a fixed-length vector using TF-IDF vectorization. The length of the output
vector is equal to 1024. Each column corresponds to one of the top word n-grams
where n is in the range [1, 2].

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

for col in {'type', 'author'}:
    vectorizer = Pipeline(steps=[
        ("imputer", SimpleImputer(missing_values=None, strategy="constant", fill_value="")),
        # Reshape to 1D since SimpleImputer changes the shape of the input to 2D
        ("reshape", FunctionTransformer(np.reshape, kw_args={"newshape":-1})),
        ("tfidf", TfidfVectorizer(decode_error="ignore", ngram_range = (1, 2), max_features=1024))])

    transformers.append((f"text_{col}", vectorizer, [col]))

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

### Feature standardization
Scale all feature columns to be centered around zero with unit variance.

In [ ]:
from sklearn.preprocessing import StandardScaler

standardizer = StandardScaler()

## Train - Validation - Test Split
Split the input data into 3 sets:
- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters of the model)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [ ]:
df_loaded.columns

Out[35]: Index(['author', 'published', 'title', 'text', 'language', 'site_url',
 'main_img_url', 'type', 'title_without_stopwords',
 'text_without_stopwords', 'hasImage', 'label'],
 dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split

split_X = df_loaded.drop([target_col], axis=1)
split_y = df_loaded[target_col]

# Split out train data
X_train, split_X_rem, y_train, split_y_rem = train_test_split(split_X, split_y, train_size=0.6, random_state=799811440, stratify=split_y)

# Split remaining data equally for validation and test
X_val, X_test, y_val, y_test = train_test_split(split_X_rem, split_y_rem, test_size=0.5, random_state=799811440, stratify=split_y_rem)

## Train classification model
- Log relevant metrics to MLflow to track runs
- All the runs are logged under [this MLflow experiment](#mlflow/experiments/406583024052808/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Change the model parameters and re-run the training cell to log a different trial to the MLflow experiment
- To view the full list of tunable hyperparameters, check the output of the cell below

In [ ]:
from xgboost import XGBClassifier

help(XGBClassifier)

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 XGBClassifier(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic', use_label_encoder: bool = True, **kwargs: Any) -> None
 
 Implementation of the scikit-learn API for XGBoost classification.
 
 
 Parameters
 ----------
 
 n_estimators : int
 Number of boosting rounds.
 use_label_encoder : bool
 (Deprecated) Use the label encoder from scikit-learn to encode the labels. For new
 code, we recommend that you set this parameter to False.
 
 max_depth : Optional[int]
 Maximum tree depth for base learners.
 learning_rate : Optional[float]
 Boosting learning rate (xgb's "eta")
 verbosity : Optional[int]
 The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 objective : typing.Union[str, typing.Callable[[numpy.ndarray, numpy.ndarray], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]
 Specify the learning task and the corresponding learning objective or
 a custom objective function to be used (see note below).
 booster: Optional[str]
 Specify which booster to use: gbtree, gblinear or dart.
 tree_method: Optional[str]
 Specify which tree method to use. Default to auto. If this parameter
 is set to default, XGBoost will choose the most conservative option
 available. It's recommended to study this option from the parameters
 document: https://xgboost.readthedocs.io/en/latest/treemethod.html.
 n_jobs : Optional[int]
 Number of parallel threads used to run xgboost. When used with other Scikit-Learn
 algorithms like grid search, you may choose which algorithm to parallelize and
 balance the threads. Creating thread contention will significantly slow down both
 algorithms.
 gamma : Optional[float]
 Minimum loss reduction required to make a further partition on a leaf
 node of the tree.
 min_child_weight : Optional[float]
 Minimum sum of instance weight(hessian) needed in a child.
 max_delta_step : Optional[float]
 Maximum delta step we allow each tree's weight estimation to be.
 subsample : Optional[float]
 Subsample ratio of the training instance.
 colsample_bytree : Optional[float]
 Subsample ratio of columns when constructing each tree.
 colsample_bylevel : Optional[float]
 Subsample ratio of columns for each level.
 colsample_bynode : Optional[float]
 Subsample ratio of columns for each split.
 reg_alpha : Optional[float]
 L1 regularization term on weights (xgb's alpha).
 reg_lambda : Optional[float]
 L2 regularization term on weights (xgb's lambda).
 scale_pos_weight : Optional[float]
 Balancing of positive and negative weights.
 base_score : Optional[float]
 The initial prediction score of all instances, global bias.
 random_state : Optional[Union[numpy.random.RandomState, int]]
 Random number seed.
 
 .. note::
 
 Using gblinear booster with shotgun updater is nondeterministic as
 it uses Hogwild algorithm.
 
 missing : float, default np.nan
 Value in the data which needs to be present as a missing value.
 num_parallel_tree: Optional[int]
 Used for boosting random forest.
 monotone_constraints : Optional[Union[Dict[str, int], str]]
 Constraint of variable monotonicity. See tutorial for more
 information.
 interaction_constraints : Optional[Union[str, List[Tuple[str]]]]
 Constraints for interaction representing permitted interactions. The
 constraints must be specified in the form of a nest list, e.g. [[0, 1],
 [2, 3, 4]], where each inner list is a group of indices of features
 that are allowed to interact with each other. See tutorial for more
 information
 importance_type: Optional[str]
 The feature importance type for the feature_importances\_ property:
 
 * For tree model, it's either "gain", "weight", "cover", "total_gain" or
 "total_cover".
 * For linear model, only "weight" is defined and it's the normalized coefficients
 without bias.
 
 gpu_id : Optional[int]
 Device ordinal.
 validate_parameters : Optional[bool

In [ ]:
import mlflow
import sklearn
from sklearn import set_config
from sklearn.pipeline import Pipeline

set_config(display="diagram")

xgbc_classifier = XGBClassifier(
  colsample_bytree=0.7324555878929649,
  learning_rate=0.007636627530856404,
  max_depth=7,
  min_child_weight=6,
  n_estimators=106,
  n_jobs=100,
  subsample=0.6972187716458148,
  verbosity=0,
  random_state=799811440,
)

model = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
    ("classifier", xgbc_classifier),
])

# Create a separate pipeline to transform the validation dataset. This is used for early stopping.
pipeline = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
])

mlflow.sklearn.autolog(disable=True)
X_val_processed = pipeline.fit_transform(X_val, y_val)

model

Out[14]:

Pipeline Pipeline(steps=[('column_selector',
 ColumnSelector(cols=['text_without_stopwords', 'published',
 'language', 'main_img_url', 'site_url',
 'hasImage', 'title_without_stopwords',
 'text', 'title', 'type', 'author'])),
 ('preprocessor',
 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('onehot',
 OneHotEncoder(handle_unknown='ignore'),
 ['publish...
 learning_rate=0.007636627530856404,
 max_delta_step=None, max_depth=7,
 min_child_weight=6, missing=nan,
 monotone_constraints=None, n_estimators=106,
 n_jobs=100, num_parallel_tree=None,
 predictor=None, random_state=799811440,
 reg_alpha=None, reg_lambda=None,
 scale_pos_weight=None,
 subsample=0.6972187716458148, tree_method=None,
 validate_parameters=None, verbosity=0))]) ColumnSelector ColumnSelector(cols=['text_without_stopwords', 'published', 'language',
 'main_img_url', 'site_url', 'hasImage',
 'title_without_stopwords', 'text', 'title', 'type',
 'author']) preprocessor: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('onehot',
 OneHotEncoder(handle_unknown='ignore'),
 ['published', 'language', 'site_url',
 'hasImage', 'title',
 'title_without_stopwords',
 'text_without_stopwords']),
 ('text_hasher',
 Pipeline(steps=[('imputer',
 SimpleImputer(fill_value='',
 missing_values=None,
 strategy='constant')),
 ('text_...
 max_features=1024,
 ngram_range=(1,
 2)))]),
 ['type']),
 ('text_author',
 Pipeline(steps=[('imputer',
 SimpleImputer(fill_value='',
 missing_values=None,
 strategy='constant')),
 ('reshape',
 FunctionTransformer(func= ,
 kw_args={'newshape': -1})),
 ('tfidf',
 TfidfVectorizer(decode_error='ignore',
 max_features=1024,
 ngram_range=(1,
 2)))]),
 ['author'])]) onehot ['published', 'language', 'site_url', 'hasImage', 'title', 'title_without_stopwords', 'text_without_stopwords'] OneHotEncoder OneHotEncoder(handle_unknown='ignore') text_hasher ['text'] SimpleImputer SimpleImputer(fill_value='', missing_values=None, strategy='constant') FeatureHasher FeatureHasher(input_type='string', n_features=1024) main_img_url_hasher ['main_img_url'] SimpleImputer SimpleImputer(fill_value='', missing_values=None, strategy='constant') FeatureHasher FeatureHasher(input_type='string', n_features=1024) text_type ['type'] SimpleImputer SimpleImputer(fill_value='', missing_values=None, strategy='constant') FunctionTransformer FunctionTransformer(func= ,
 kw_args={'newshape': -1}) TfidfVectorizer TfidfVectorizer(decode_error='ignore', max_features=1024, ngram_range=(1, 2)) text_author ['author'] SimpleImputer SimpleImputer(fill_value='', missing_values=None, strategy='constant') FunctionTransformer FunctionTransformer(func= ,
 kw_args={'newshape': -1}) TfidfVectorizer TfidfVectorizer(decode_error='ignore', max_features=1024, ngram_range=(1, 2)) remainder [['text_without_stopwords', 'published', 'language', 'main_img_url', 'site_url', 'hasImage', 'title_without_stopwords', 'text', 'title', 'type', 'author']] passthrough passthrough StandardScaler StandardScaler() XGBClassifier XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
 colsample_bynode=None, colsample_bytree=0.7324555878929649,
 enable_categorical=False, gamma=None, gpu_id=None,
 importance_type=None, interaction_constraints=None,
 learning_rate=0.007636627530856404, max_delta_step=None,
 max_depth=7, min_child_weight=6, missing=nan,
 monotone_constraints=None, n_estimators=106, n_jobs=100,
 num_parallel_tree=None, predictor=None, random_state=799811440,
 reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
 subsample=0.6972187716458148, tree_method=None,
 validate_parameters=None, verbosity=0)

In [ ]:
# Enable automatic logging of input samples, metrics, parameters, and models
mlflow.sklearn.autolog(log_input_examples=True, silent=True)

with mlflow.start_run(run_name="xgboost") as mlflow_run:
    model.fit(X_train, y_train, classifier__early_stopping_rounds=5, classifier__eval_set=[(X_val_processed,y_val)], classifier__verbose=False)
    
    # Training metrics are logged by MLflow autologging
    # Log metrics for the validation set
    xgbc_val_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_val, y_val, prefix="val_")

    # Log metrics for the test set
    xgbc_test_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_test, y_test, prefix="test_")

    # Display the logged metrics
    xgbc_val_metrics = {k.replace("val_", ""): v for k, v in xgbc_val_metrics.items()}
    xgbc_test_metrics = {k.replace("test_", ""): v for k, v in xgbc_test_metrics.items()}
    display(pd.DataFrame([xgbc_val_metrics, xgbc_test_metrics], index=["validation", "test"]))

/databricks/python/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
 warnings.warn(label_encoder_deprecation_msg, UserWarning)

precision_score,recall_score,f1_score,accuracy_score,log_loss,roc_auc_score,score
0.9739393743886414,0.9737470167064439,0.9738315742234848,0.9737470167064439,0.6431112029102936,0.990724989706254,0.9737470167064439
0.9601811036156819,0.9594272076372315,0.9595589291413146,0.9594272076372315,0.6588218042844805,0.9930108232931626,0.9594272076372315


In [ ]:
# Patch requisite packages to the model environment YAML for model serving
import os
import shutil
import uuid
import yaml

None

import xgboost
from mlflow.tracking import MlflowClient

xgbc_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], str(uuid.uuid4())[:8])
os.makedirs(xgbc_temp_dir)
xgbc_client = MlflowClient()
xgbc_model_env_path = xgbc_client.download_artifacts(mlflow_run.info.run_id, "model/conda.yaml", xgbc_temp_dir)
xgbc_model_env_str = open(xgbc_model_env_path)
xgbc_parsed_model_env_str = yaml.load(xgbc_model_env_str, Loader=yaml.FullLoader)

xgbc_parsed_model_env_str["dependencies"][-1]["pip"].append(f"xgboost=={xgboost.__version__}")

with open(xgbc_model_env_path, "w") as f:
  f.write(yaml.dump(xgbc_parsed_model_env_str))
xgbc_client.log_artifact(run_id=mlflow_run.info.run_id, local_path=xgbc_model_env_path, artifact_path="model")
shutil.rmtree(xgbc_temp_dir)

## Feature importance

SHAP is a game-theoretic approach to explain machine learning models, providing a summary plot
of the relationship between features and model output. Features are ranked in descending order of
importance, and impact/color describe the correlation between the feature and the target variable.
- Generating SHAP feature importance is a very memory intensive operation, so to ensure that AutoML can run trials without
  running out of memory, we disable SHAP by default.<br />
  You can set the flag defined below to `shap_enabled = True` and re-run this notebook to see the SHAP plots.
- To reduce the computational overhead of each trial, a single example is sampled from the validation set to explain.<br />
  For more thorough results, increase the sample size of explanations, or provide your own examples to explain.
- SHAP cannot explain models using data with nulls; if your dataset has any, both the background data and
  examples to explain will be imputed using the mode (most frequent values). This affects the computed
  SHAP values, as the imputed samples may not match the actual data distribution.

For more information on how to read Shapley values, see the [SHAP documentation](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html).

In [ ]:
# Set this flag to True and re-run the notebook to see the SHAP plots
shap_enabled = True

In [ ]:
if shap_enabled:
    from shap import KernelExplainer, summary_plot
    # SHAP cannot explain models using data with nulls.
    # To enable SHAP to succeed, both the background data and examples to explain are imputed with the mode (most frequent values).
    mode = X_train.mode().iloc[0]

    # Sample background data for SHAP Explainer. Increase the sample size to reduce variance.
    train_sample = X_train.sample(n=min(100, len(X_train.index))).fillna(mode)

    # Sample a single example from the validation set to explain. Increase the sample size and rerun for more thorough results.
    example = X_val.sample(n=1).fillna(mode)

    # Use Kernel SHAP to explain feature importance on the example from the validation set.
    predict = lambda x: model.predict_proba(pd.DataFrame(x, columns=X_train.columns))
    explainer = KernelExplainer(predict, train_sample, link="logit")
    shap_values = explainer.shap_values(example, l1_reg=False)
    summary_plot(shap_values, example, class_names=model.classes_)

0%| | 0/1 [00:00<?, ?it/s]

## Inference
[The MLflow Model Registry](https://docs.databricks.com/applications/mlflow/model-registry.html) is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance. The snippets below show how to add the model trained in this notebook to the model registry and to retrieve it later for inference.

> **NOTE:** The `model_uri` for the model already trained in this notebook can be found in the cell below

### Register to Model Registry
```
model_name = "Example"

model_uri = f"runs:/{ mlflow_run.info.run_id }/model"
registered_model_version = mlflow.register_model(model_uri, model_name)
```

### Load from Model Registry
```
model_name = "Example"
model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
model.predict(input_X)
```

### Load model without registering
```
model_uri = f"runs:/{ mlflow_run.info.run_id }/model"

model = mlflow.pyfunc.load_model(model_uri)
model.predict(input_X)
```

In [ ]:
# model_uri for the generated model
print(f"runs:/{ mlflow_run.info.run_id }/model")

runs:/51c0348482e042ea8e4b7983ab6bff99/model

### Loading model to make prediction

In [ ]:
model_uri = f"runs:/51c0348482e042ea8e4b7983ab6bff99/model"

model = mlflow.pyfunc.load_model(model_uri)
#model.predict(input_X)

In [ ]:
import pandas as pd
data = {'author': {0: 'bigjim.com'},
 'published': {0: '2016-10-27T18:05:26.351+03:00'},
 'title': {0: 'aliens are coming to invade earth'},
 'text': {0: 'aliens are coming to invade earth'},
 'language': {0: 'english'},
 'site_url': {0: 'cnn.com'},
 'main_img_url': {0: 'https://2.bp.blogspot.com/-0mdp0nZiwMI/UYwYvexmW2I/AAAAAAAAVQM/7C_X5WRE_mQ/w1200-h630-p-nu/Edison-Stock-Ticker.jpg'},
 'type': {0: 'bs'},
 'title_without_stopwords': {0: 'aliens are coming to invade earth'},
 'text_without_stopwords': {0: 'aliens are coming to invade earth'},
 'hasImage': {0: 1.0}}
df = pd.DataFrame(data=data)
df.head()

Out[4]:

,author,published,title,text,language,site_url,main_img_url,type,title_without_stopwords,text_without_stopwords,hasImage
0,bigjim.com,2016-10-27T18:05:26.351+03:00,aliens are coming to invade earth,aliens are coming to invade earth,english,cnn.com,https://2.bp.blogspot.com/-0mdp0nZiwMI/UYwYvex...,bs,aliens are coming to invade earth,aliens are coming to invade earth,1.0


In [ ]:
model.predict(df)

Out[45]: array(['Fake'], dtype=object)